# Welcome to Antidote

## Introduction

In this tutorial, you will be introduced to Antidote Databases. You can test and see how an eventually consistent database works. We will guide you through the CRDT data types supported by Antidote, transactions, and how Antidote resolves inconsistencies. For more information, see http://syncfree.github.io/antidote/.

> Note that this Jupyter notebook is based on Antidote Java API, but it is tailored for tuturial purpose. As a result, this tutorial might not reflect the full capabilities of Antidote. To find out how to use the API, please refer to the __[official Antidote Java API](https://www.javadoc.io/doc/eu.antidotedb/antidote-java-client/0.1.0)__.

## Antidote data types

Antidote supports several Conflict-free replicated data types (CRDT). Each type handles inconsistencies differently. In this section, we show some examples of how you can use these data types. The data types supported by Antidote include:
- __Number types__
    - Counter
    - Fat counter
    - Integer
- __Registers__
    - Last-writer wins register
    - Multi-value register
- __Sets__
    - Add-wins set (or Observed-remove set)
    - Remove-win set
- __Maps__
    - Grow-only map
    - Add-wins map
    - Remove-resets map

First of all, you need to make sure that Antidote service is up and running.

In [1]:
init()

22:36:59 Antidote session created. Connected to Antidote node 1.

In Antidote, an object of any data types consists of 2 parts: a key and a value. You need a key in order to update/remove/reset/read your data object in an Antidote bucket. A bucket is an internal data structure used by Antidote where objects are stored. In this tutorial, we have 1 bucket which uses a unique session id as its key. An object key should be unique across the bucket, regardless of data types.

### Counter

Counter is an integer object with some operations. First, we need to create a key for our counter object. We assign "c1" as its key.

In [116]:
counter_key = getCounterKey("c1")

COUNTER_c1

The counter datatype is simply an integer which can be incremented or decremented. Let's try incrementing our c1 counter by 10, and decrementing it by 1.

In [117]:
applyUpdate(incrementCounter(counter_key, 10))
applyUpdate(incrementCounter(counter_key, -1))

22:45:20 Updated key 'COUNTER_c1' on Antidote 3

Reading a counter object returns the aggregated value from all received operations. In our example, the c1 counter should now be equal to 9.

In [118]:
read(counter_key)

18

Fat counter and Integer is similar to Counter, except that, in Fat counter, you can reset the counter to its inital state, while in Integer, you can assign a value to it.

### Register

Register can store a single-valued object. We use a register object to store string in the following example.

In [119]:
register_key = getLWRegisterKey("r1")

LWWREG_r1

In [120]:
applyUpdate(assignLWRegister(register_key, "Hello"))

22:45:20 Updated key 'LWWREG_r1' on Antidote 3

In [121]:
read(register_key)

Hello

### Set

Set can store a collection of values of the same type. This notebook supports only a Set of Strings.

In [122]:
set_key = getSetKey("s1")

ORSET_s1

In [123]:
applyUpdate(addToSet(set_key, "A","B","C","D"))

22:45:20 Updated key 'ORSET_s1' on Antidote 3

In [124]:
read(set_key)

[A, B, C, D, initial, original]

In [125]:
applyUpdate(removeFromSet(set_key, "C","D"))

22:45:20 Updated key 'ORSET_s1' on Antidote 3

In [126]:
read(set_key)

[A, B, initial, original]

### Map

Map can store a collection of values of different Antidote data types. In the following example, we create a map which stores 2 element objects: an integer, and a fat counter object.

In [127]:
map_key = getAWMapKey("m1")

AWMAP_m1

In [128]:
integer_key = getIntegerKey("i1")

INTEGER_i1

In [129]:
fatcounter_key = getFatCounterKey("f1")

FATCOUNTER_f1

To update a map, we actually update element objects of the map.

In [130]:
applyUpdate(updateAWMap(map_key, assignInteger(integer_key, 1)
                               , incrementFatCounter(fatcounter_key, 2)))

22:45:20 Updated key 'AWMAP_m1' on Antidote 3

To read from a map, we also need to specify the key of the element object.

In [131]:
readFromMap(map_key, integer_key)

1

In [132]:
readFromMap(map_key, fatcounter_key)

4

In [133]:
applyUpdate(removeFromAWMap(map_key, integer_key))

22:45:21 Updated key 'AWMAP_m1' on Antidote 3

In [134]:
readFromMap(map_key, integer_key)

Cell returned null.

In [135]:
readFromMap(map_key, fatcounter_key)

4

## Transactions

So far, we use `applyUpdate()` to update the Antidote objects, but you can also group several updates to create an atomic transaction. Let's see an example below. We create a transaction that will assign a value to x and incremrent y.

In [136]:
x = getIntegerKey("x")

INTEGER_x

In [137]:
y = getCounterKey("y")

COUNTER_y

In [138]:
tx = startTransaction()

eu.antidotedb.client.InteractiveTransaction@a0b8e32

In [139]:
addToTransaction(tx, incrementInteger(x, 10))

22:45:21 Added key 'INTEGER_x' to transaction

In [140]:
addToTransaction(tx, incrementCounter(y, 1))

22:45:21 Added key 'COUNTER_y' to transaction

The updates will take effect only after a transaction is committed.

In [141]:
commitTransaction(tx)

22:45:21 Transaction committed on Antidote 3

In [142]:
read(x)

12

In [143]:
read(y)

2

## Data replication

In Antidote, replicated databases are eventually consistent. Each data type resolves the consistencies differently. In this section, we will demonstrate this by using 2 replicated Antidote nodes: _Antidote1_ and _Antidote2_. By default, this notebook connects to Antidote1.

<img src="images/Antidote-Notebook-diagram.png">

Both Antidote1 and Antidote2 are currently connected. Any updates to either one of them will be replicated to the other one.

In [144]:
x = getIntegerKey("x")

INTEGER_x

In [145]:
applyUpdate(assignInteger(x, 2))

22:45:21 Updated key 'INTEGER_x' on Antidote 3

In [146]:
read(x)

2

Now we switch to Antidote2 to see if x is 2 as well.

In [147]:
switchAntidote(2)

22:45:21 Connected to Antidote 2.

In [148]:
read(x)

2

## Inconsistency resolution

In this section, we will see how Antidote resolves conflicts when updates are done on disconnected replicas.

### Last-writer-wins

The final value of the object is the value updated at the latest timestamp. The Last-writer-wins register uses this mechanism.

In [149]:
switchAntidote(1)
lww_key = getLWRegisterKey("lww1")
applyUpdate(assignLWRegister(lww_key, "initial"))
read(lww_key)

initial

In [150]:
disconnectAntidotes()

22:45:22 Disconnecting Antidote nodes.

`disconnectAntidotes()` and `connectAntidotes()` simulates a disconnection and connection between the 2 Antidote nodes. <img src="images/Antidote-Notebook-diagram-disconnected.png">

In [151]:
switchAntidote(1)

22:45:22 Connected to Antidote 1.

In [152]:
applyUpdate(assignLWRegister(lww_key, "update at 1"))

22:45:22 Updated key 'LWWREG_lww1' on Antidote 3

In [153]:
switchAntidote(2)

22:45:22 Connected to Antidote 2.

In [154]:
applyUpdate(assignLWRegister(lww_key, "update at 2"))

22:45:22 Updated key 'LWWREG_lww1' on Antidote 3

In [155]:
connectAntidotes()

22:45:22 Connecting Antidote nodes.

The expected value stored in key lww1 is `"update at 2"` since it is the last update performed on key lww1.

In [156]:
read(lww_key)

update at 2

### Multi-value

Reading a Multi-value Register (MV-Register) returns a list of all concurrently assigned values, which have not been overridden by other assignments. The returned list is ordered by the lexicographic order on the byte representation of the value.
In the initial state reading the register returns the empty list.

In [2]:
switchAntidote(1)
mv_key = getMVRegisterKey("mv1")
applyUpdate(assignMVRegister(mv_key, "initial"))
read(mv_key)

[initial]

In [3]:
disconnectAntidotes()

22:37:07 Disconnecting Antidote nodes.

In [4]:
switchAntidote(1)

22:37:08 Connected to Antidote 1.

In [5]:
applyUpdate(assignMVRegister(mv_key, "update at 1"))

22:37:09 Updated key 'MVREG_mv1' on Antidote 1

In [6]:
switchAntidote(2)

22:37:12 Antidote 2 session created.  Connected to Antidote 2.

In [7]:
applyUpdate(assignMVRegister(mv_key, "update at 2 - first try"))
applyUpdate(assignMVRegister(mv_key, "update at 2 - second try"))

22:37:14 Updated key 'MVREG_mv1' on Antidote 2

In [8]:
connectAntidotes()

22:37:15 Connecting Antidote nodes.

The expected values stored on key mv1 is `["update at 1","update at 2 - second try"]` since both are the last update on different Antidote nodes during disconnection.

In [12]:
read(mv_key)

[update at 1, update at 2 - second try]

### Add-wins

Add-operations win over concurrent remove-operations. A remove-operation will only “overwrite” the add-operations that happened before it. More precisely, an element is in the set, if there is an add-operation for the element which is not followed by a remove-operation for the same element (or a reset-operation).

In [13]:
switchAntidote(1)
aw_key = getSetKey("aw1")
applyUpdate(addToSet(aw_key, "initial"))
read(aw_key)

[initial]

In [14]:
disconnectAntidotes()

22:37:46 Disconnecting Antidote nodes.

In [15]:
switchAntidote(1)

22:37:47 Connected to Antidote 1.

In [16]:
applyUpdate(addToSet(aw_key, "A"))

22:37:48 Updated key 'ORSET_aw1' on Antidote 1

In [17]:
switchAntidote(2)

22:37:49 Connected to Antidote 2.

In [18]:
applyUpdate(removeFromSet(aw_key, "A"))

22:37:50 Updated key 'ORSET_aw1' on Antidote 2

In [19]:
connectAntidotes()

22:37:53 Connecting Antidote nodes.

The expected values stored on key aw_key are `["initial","A"]`. There was an attempt to add and remove "A" during disconnection, "A" is included in the final values since this is an add-wins mechanism.

In [23]:
read(aw_key)

[A, initial]

### Remove-wins

Remove-operations win over concurrent add-operations. Here, an add-operation overwrites only the remove-operations that happened before it. More precisely, an element is removed, if there is a remove-operation for the element which is not followed by an add-operation for the same element. An element is in the set, if it has been added and not removed.

In [24]:
switchAntidote(1)
rw_key = getSetKey("rw1")
applyUpdate(addToSet(rw_key, "initial"))
read(rw_key)

[initial]

In [25]:
disconnectAntidotes()

22:38:13 Disconnecting Antidote nodes.

In [26]:
switchAntidote(1)

22:38:13 Connected to Antidote 1.

In [27]:
applyUpdate(addToSet(rw_key, "A"))

22:38:14 Updated key 'ORSET_rw1' on Antidote 1

In [28]:
switchAntidote(2)

22:38:15 Connected to Antidote 2.

In [29]:
applyUpdate(removeFromSet(rw_key, "A"))

22:38:15 Updated key 'ORSET_rw1' on Antidote 2

In [30]:
connectAntidotes()

22:38:16 Connecting Antidote nodes.

The expected value stored on key aw_key is only `["initial"]`. There was an attempt to add and remove "A" during disconnection, but "A" is not included in the final values since this is a remove-wins mechanism.

In [34]:
read(rw_key)

[A, initial]

### Remove-reset